# Web Scraping for Question-Answer Extraction

This notebook demonstrates how to scrape web pages for question-answer pairs, excluding specific sections and classes using Python. It utilizes `requests` and `BeautifulSoup` libraries for HTTP requests and HTML parsing.

The code performs the following steps:

1. **Fetch URLs from Sitemap**:
   - Requests the sitemap URL to get a list of URLs to scrape.

2. **Extract Question-Answer Pairs**:
   - Defines a function to extract question-answer pairs from the HTML content.
   - Removes specific sections and elements that should be excluded based on their class names.
   - Collects questions and their corresponding answers from the page.

3. **Process Each URL**:
   - Iterates through the list of URLs.
   - Requests each URL and processes the HTML content.
   - Saves the extracted question-answer pairs to text files.


In [ ]:
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import re

# 1. Fetch URLs from Sitemap
sitemap_url = 'https://www.telekom.de/ueber-das-unternehmen/robots/sitemap'
print(f"Requesting sitemap URL: {sitemap_url}")
response = requests.get(sitemap_url)

# Parse XML sitemap
soup = BeautifulSoup(response.content, 'xml')
print("Processing sitemap XML...")

# Extract URLs
urls = [url.text for url in soup.find_all('loc') if url is not None]
print(f"{len(urls)} URLs found.")

def extract_question_answer(soup):
    question_answer_pairs = []
    
    # Function to get text from element
    def get_text_from_element(element):
        text = ''
        for p in element.find_all('p'):
            text += p.get_text(strip=True) + '\n'
        for ul in element.find_all('ul'):
            for li in ul.find_all('li'):
                text += f"• {li.get_text(strip=True)}\n"
        return text.strip()

    # Define excluded classes
    excluded_classes = [
        "chf-navigation-bar",
        "direct-access-container",
        "direct-access-content",
        "collection-wrapper collection collection-standard",
        "collection-wrapper collection collection-standard l-outer l-outer--solutionPage"
    ]
    
    # Function to remove excluded elements
    def remove_excluded_elements(soup):
        for class_name in excluded_classes:
            for element in soup.find_all(class_=class_name):
                if element:
                    element.decompose()
    
    # Function to remove nested excluded elements
    def remove_nested_excluded_elements(soup):
        for element in soup.find_all(True):
            if isinstance(element, BeautifulSoup):
                classes = element.get('class', [])
                if classes and any(cls in ' '.join(classes) for cls in excluded_classes):
                    element.decompose()
    
    remove_excluded_elements(soup)
    remove_nested_excluded_elements(soup)

    # Extract questions and answers
    questions = soup.find_all(['h1', 'h2', 'h3'])
    
    for question in questions:
        question_text = question.get_text(strip=True)
        if question_text.endswith('?'):
            answer_text = ''
            # Check for answer in next div
            next_div = question.find_next('div', class_='outerRichtextDiv')
            if next_div:
                answer_text = get_text_from_element(next_div)
            if not answer_text:
                next_div = question.find_next('div')
                if next_div and not any(cls in ' '.join(next_div.get('class', [])) for cls in excluded_classes):
                    answer_text = get_text_from_element(next_div)
            if answer_text:
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    return question_answer_pairs

# 3. Process each URL
output_dir = Path("data")
output_dir.mkdir(parents=True, exist_ok=True)  # Create directory

for idx, url in enumerate(urls, 1):
    print(f"Processing URL {idx}/{len(urls)}: {url}")
    
    try:
        response = requests.get(url, allow_redirects=False)
        
        if response.status_code == 301 or response.status_code == 302:
            print("   Redirect detected, checking URL.")
            final_url = response.headers.get('Location')
            if final_url:
                response = requests.get(final_url)
        elif response.status_code == 200:
            response = requests.get(url)
        else:
            print("   Invalid URL or access problem.")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        qa_pairs = extract_question_answer(soup)
        print(f"   {len(qa_pairs)} question-answer pairs found.")
        
        if qa_pairs:
            file_name = re.sub(r'\W+', '_', url) + ".txt"
            output_file = output_dir / file_name
            
            with open(output_file, "w", encoding="utf-8") as file:
                file.write(f"Source URL: {url}\n\n")
                for idx, qa in enumerate(qa_pairs, 1):
                    file.write(f"{idx}. Question: {qa['question']}\n   Answer: {qa['answer']}\n\n")
            
            print(f"   Results saved to '{output_file}'.")
        else:
            print("   No question-answer pairs found, file will not be created.")
    
    except requests.exceptions.RequestException as e:
        print(f"   Error occurred: {e}")

print("Processing completed! Results for pages with question-answer pairs saved in 'data' directory.")


### YEDEK OLARAK TUTUYORUM, GÜNCEL DEGIL! Accordion listeleri ekledim koda, 1200 cekiyordu simdi daha fazla olmali daha sonra kontrol et

In [ ]:
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import re

# 1. Fetch URLs from Sitemap
sitemap_url = 'https://www.telekom.de/ueber-das-unternehmen/robots/sitemap'
print(f"Requesting sitemap URL: {sitemap_url}")
response = requests.get(sitemap_url)

# Parse XML sitemap
soup = BeautifulSoup(response.content, 'xml')
print("Processing sitemap XML...")

# Extract URLs
urls = [url.text for url in soup.find_all('loc') if url is not None]
print(f"{len(urls)} URLs found.")

# Function to extract question-answer pairs from the page, including accordion lists
def extract_question_answer(soup):
    question_answer_pairs = []

    # Function to get text from element
    def get_text_from_element(element):
        text = ''
        for p in element.find_all('p'):
            text += p.get_text(strip=True) + '\n'
        for ul in element.find_all('ul'):
            for li in ul.find_all('li'):
                text += f"• {li.get_text(strip=True)}\n"
        return text.strip()

    # Function to extract accordion items
    def extract_accordion_items(accordion_list):
        for item in accordion_list.find_all('li', class_='accordion-item'):
            question = item.find('p', class_='accordion-item__title')
            answer = item.find('div', class_='accordion-item__content')
            if question and answer:
                question_text = question.get_text(strip=True)
                answer_text = get_text_from_element(answer)
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    # Define excluded classes
    excluded_classes = [
        "chf-navigation-bar",
        "direct-access-container",
        "direct-access-content",
        "collection-wrapper collection collection-standard",
        "collection-wrapper collection collection-standard l-outer l-outer--solutionPage"
    ]
    
    # Function to remove excluded elements
    def remove_excluded_elements(soup):
        for class_name in excluded_classes:
            for element in soup.find_all(class_=class_name):
                if element:
                    element.decompose()
    
    # Function to remove nested excluded elements
    def remove_nested_excluded_elements(soup):
        for element in soup.find_all(True):
            if isinstance(element, BeautifulSoup):
                classes = element.get('class', [])
                if classes and any(cls in ' '.join(classes) for cls in excluded_classes):
                    element.decompose()
    
    remove_excluded_elements(soup)
    remove_nested_excluded_elements(soup)

    # Extract questions and answers from headings and next divs
    questions = soup.find_all(['h1', 'h2', 'h3'])
    for question in questions:
        question_text = question.get_text(strip=True)
        if question_text.endswith('?'):
            answer_text = ''
            next_div = question.find_next('div', class_='outerRichtextDiv')
            if next_div:
                answer_text = get_text_from_element(next_div)
            if not answer_text:
                next_div = question.find_next('div')
                if next_div and not any(cls in ' '.join(next_div.get('class', [])) for cls in excluded_classes):
                    answer_text = get_text_from_element(next_div)
            if answer_text:
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    # Extract questions and answers from accordion lists
    accordion_list = soup.find('ul', class_='accordion-list')
    if accordion_list:
        extract_accordion_items(accordion_list)

    return question_answer_pairs

# 3. Process each URL
output_dir = Path("data")
output_dir.mkdir(parents=True, exist_ok=True)  # Create directory

for idx, url in enumerate(urls, 1):
    print(f"Processing URL {idx}/{len(urls)}: {url}")
    
    try:
        response = requests.get(url, allow_redirects=False)
        
        if response.status_code == 301 or response.status_code == 302:
            print("   Redirect detected, checking URL.")
            final_url = response.headers.get('Location')
            if final_url:
                response = requests.get(final_url)
        elif response.status_code == 200:
            response = requests.get(url)
        else:
            print("   Invalid URL or access problem.")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract question-answer pairs
        qa_pairs = extract_question_answer(soup)
        print(f"   {len(qa_pairs)} question-answer pairs found.")
        
        if qa_pairs:  # Only create file if there are question-answer pairs
            file_name = re.sub(r'\W+', '_', url) + ".txt"
            output_file = output_dir / file_name
            
            with open(output_file, "w", encoding="utf-8") as file:
                file.write(f"Source URL: {url}\n\n")
                for idx, qa in enumerate(qa_pairs, 1):
                    file.write(f"{idx}. Question: {qa['question']}\n   Answer: {qa['answer']}\n\n")
            
            print(f"   Results saved to '{output_file}'.")
        else:
            print("   No question-answer pairs found, file will not be created.")
    
    except requests.exceptions.RequestException as e:
        print(f"   Error occurred: {e}")

print("Processing completed! Results for pages with question-answer pairs saved in 'data' directory.")


## LAST VERSION! Gpt biraz daha iyilestirdi, cok daha hizli calisiyor!
### Navigation yok!!

In [1]:
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import re
import logging
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to create a session with retry logic
def create_session():
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# 1. Fetch URLs from Sitemap
sitemap_url = 'https://www.telekom.de/ueber-das-unternehmen/robots/sitemap'
logger.info(f"Requesting sitemap URL: {sitemap_url}")
session = create_session()

try:
    response = session.get(sitemap_url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    logger.error(f"Error fetching sitemap: {e}")
    exit(1)

# Parse XML sitemap
soup = BeautifulSoup(response.content, 'xml')
logger.info("Processing sitemap XML...")

# Extract URLs
urls = [url.text for url in soup.find_all('loc') if url is not None]
logger.info(f"{len(urls)} URLs found.")

# Function to extract question-answer pairs from the page
def extract_question_answer(soup):
    question_answer_pairs = []

    def get_text_from_element(element):
        text = ''
        for p in element.find_all('p'):
            text += p.get_text(strip=True) + '\n'
        for ul in element.find_all('ul'):
            for li in ul.find_all('li'):
                text += f"• {li.get_text(strip=True)}\n"
        return text.strip()

    def extract_accordion_items(accordion_list):
        for item in accordion_list.find_all('li', class_='accordion-item'):
            question = item.find('p', class_='accordion-item__title')
            answer = item.find('div', class_='accordion-item__content')
            if question and answer:
                question_text = question.get_text(strip=True)
                answer_text = get_text_from_element(answer)
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    excluded_classes = [
        "chf-navigation-bar",
        "direct-access-container",
        "direct-access-content",
        "collection-wrapper collection collection-standard",
        "collection-wrapper collection collection-standard l-outer l-outer--solutionPage"
    ]
    
    def remove_excluded_elements(soup):
        for class_name in excluded_classes:
            for element in soup.find_all(class_=class_name):
                if element:
                    element.decompose()

    def remove_nested_excluded_elements(soup):
        for element in soup.find_all(True):
            if isinstance(element, BeautifulSoup):
                classes = element.get('class', [])
                if classes and any(cls in ' '.join(classes) for cls in excluded_classes):
                    element.decompose()
    
    remove_excluded_elements(soup)
    remove_nested_excluded_elements(soup)

    questions = soup.find_all(['h1', 'h2', 'h3'])
    for question in questions:
        question_text = question.get_text(strip=True)
        if question_text.endswith('?'):
            answer_text = ''
            next_div = question.find_next('div', class_='outerRichtextDiv')
            if next_div:
                answer_text = get_text_from_element(next_div)
            if not answer_text:
                next_div = question.find_next('div')
                if next_div and not any(cls in ' '.join(next_div.get('class', [])) for cls in excluded_classes):
                    answer_text = get_text_from_element(next_div)
            if answer_text:
                question_answer_pairs.append({'question': question_text, 'answer': answer_text})

    accordion_list = soup.find('ul', class_='accordion-list')
    if accordion_list:
        extract_accordion_items(accordion_list)

    return question_answer_pairs

# 3. Process each URL
output_dir = Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

for idx, url in enumerate(urls, 1):
    logger.info(f"Processing URL {idx}/{len(urls)}: {url}")
    
    try:
        response = session.get(url, allow_redirects=False)
        
        if response.status_code in {301, 302}:
            logger.info("   Redirect detected, checking URL.")
            final_url = response.headers.get('Location')
            if final_url:
                response = session.get(final_url)
        elif response.status_code != 200:
            logger.warning("   Invalid URL or access problem.")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        qa_pairs = extract_question_answer(soup)
        logger.info(f"   {len(qa_pairs)} question-answer pairs found.")
        
        if qa_pairs:
            file_name = re.sub(r'\W+', '_', url) + ".txt"
            output_file = output_dir / file_name
            
            with open(output_file, "w", encoding="utf-8") as file:
                file.write(f"Source URL: {url}\n\n")
                for idx, qa in enumerate(qa_pairs, 1):
                    file.write(f"{idx}. Question: {qa['question']}\n   Answer: {qa['answer']}\n\n")
            
            logger.info(f"   Results saved to '{output_file}'.")
        else:
            logger.info("   No question-answer pairs found, file will not be created.")
    
    except requests.exceptions.RequestException as e:
        logger.error(f"   Error occurred: {e}")

logger.info("Processing completed! Results for pages with question-answer pairs saved in 'data' directory.")


2024-08-26 21:24:16,578 - INFO - Requesting sitemap URL: https://www.telekom.de/ueber-das-unternehmen/robots/sitemap
2024-08-26 21:24:16,855 - INFO - Processing sitemap XML...
2024-08-26 21:24:16,860 - INFO - 2778 URLs found.
2024-08-26 21:24:16,861 - INFO - Processing URL 1/2778: https://www.telekom.de/apps
2024-08-26 21:24:17,003 - INFO -    Redirect detected, checking URL.
2024-08-26 21:24:17,572 - INFO -    0 question-answer pairs found.
2024-08-26 21:24:17,572 - INFO -    No question-answer pairs found, file will not be created.
2024-08-26 21:24:17,573 - INFO - Processing URL 2/2778: https://www.telekom.de/apps/meinmagenta-app
2024-08-26 21:24:18,103 - INFO -    0 question-answer pairs found.
2024-08-26 21:24:18,104 - INFO -    No question-answer pairs found, file will not be created.
2024-08-26 21:24:18,104 - INFO - Processing URL 3/2778: https://www.telekom.de/apps/voicemail
2024-08-26 21:24:18,472 - INFO -    0 question-answer pairs found.
2024-08-26 21:24:18,473 - INFO -    No